In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import torch
import torch.optim as optim
from transformers import AutoTokenizer,AutoModelForTokenClassification,DataCollatorForTokenClassification
from transformers import Trainer,TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np
import seqeval

### 1、load_dataset下载、读取数据集；
### 2、使用AutoTokenizer、AutoModelForToeknClassification加载分词器和生成模型框架

In [6]:
ds = load_dataset('doushabao4766/msra_ner_k_V3')
print(ds['train']['id'][16])
print(ds['train']['tokens'][16])
print(ds['train']['ner_tags'][16])
print(ds['train']['tokens'][16][10:13])

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

16
['去', '年', '，', '我', '们', '又', '被', '评', '为', '“', '北', '京', '市', '首', '届', '家', '庭', '藏', '书', '状', '元', '明', '星', '户', '”', '。']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['北', '京', '市']


In [7]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [8]:
tag_name = ds['train'].features['ner_tags'].feature.names
print(f'tag_name: {tag_name}')
tag_id = set()
for id in ds['train']['ner_tags']:
    tag_id.update(id)
tag_id

tag_name: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


{0, 1, 2, 3, 4, 5, 6}

### 3、根据transformers中dataset中features属性进行查看实体标签然后构造实体映射字典entite_index和实体词典集合tags

In [9]:
entites = ['O'] + list({'PER', 'LOC', 'ORG'})
tags = ['O']
for ent in entites[1:]:
    tags.append('B-' + ent.upper())
    tags.append('I-' + ent.upper())
entite_index = {entite:i for i,entite in enumerate(entites)}
entite_index

{'O': 0, 'LOC': 1, 'PER': 2, 'ORG': 3}

In [11]:
tags

['O', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER']

In [10]:
def data_input_proc(item):
    input_data = tokenizer(item['tokens'],
                           padding='max_length',
                           truncation = True,
                           max_length=512,
                           add_special_tokens=False,
                           is_split_into_words=True)  #就是中文用list变为单个汉字(这个已经是单个字了)防止合并
    # labels = [l[:512]for l in item['ner_tags']]
    input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ner_tags']]
    # input_data['labels'] = labels
    return input_data
ds1 = ds.map(data_input_proc,batched=True)

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [16]:
len(ds1['train']['input_ids'][30])

512

In [21]:
print(f'orginal: {ds1}')
ds1.set_format('torch',columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

orginal: DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})


In [85]:
for it in ds1['train']:
    print(it)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 

In [12]:
id2lbl = {i:k for i,k in enumerate(tags)}
lbl2id = {k:i for i,k in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

### 模型训练Trainer

In [87]:
args = TrainingArguments(output_dir = 'ner_train',     #训练保存的文件及模型
                         num_train_epochs=3,           #训练的epoch数量
                         save_safetensors=False,       #是否保存为safetensor
                         per_device_train_batch_size=32, #训练的batch_size
                         per_device_eval_batch_size=32,  #评估的batch size
                         report_to='tensorboard',        # 训练记录保存方式默认是all保存到wandb中
                         eval_strategy='epoch'          #每个epoch结束后评估一次
                        )

In [16]:
#可以看到我们没有做dataloader，所以在trainer中需要用DataCollatorForTokenClassification方式制造数据

#将数据元素列表转换为批次矩阵的对象，做padding、在序列首尾添加特殊标记（如 [CLS]、[SEP]），并同步调整标签
# 生成注意力掩码标签(attention mask) 、转换为张量
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,
                                                   padding=True)

In [17]:
def compute_matric(result):
     # result 是一个tuple (predicts, labels)

    #获取评估对象( 初始化评估器,Hugging Face)
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts,axis=-1)

    #准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if p != -100]for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]for ps,ls in zip(predicts,labels)]

    #计算序列标注的评估指标
    results = seqeval.compute(predicts=predicts,references=labels)
    
    # predictions：模型预测的标签序列（列表的列表）。
    # references：真实标签序列（列表的列表）。
    
    return results
    

In [18]:
trainer = Trainer(model,
                  args,
                  train_dataset=ds1['train'],
                  eval_dataset=ds1['test'],
                  data_collator=data_collator,
                  compute_metrics=compute_matric)

In [ ]:
t.train()

### 混合精度训练，所以要制造dataloder详细看步骤trainer也有单独的处理方式

In [13]:
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup

In [16]:
[name for name, params in model.named_parameters()]

['bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias',
 'bert.encoder.layer.0.attention.self.query.weight',
 'bert.encoder.layer.0.attention.self.query.bias',
 'bert.encoder.layer.0.attention.self.key.weight',
 'bert.encoder.layer.0.attention.self.key.bias',
 'bert.encoder.layer.0.attention.self.value.weight',
 'bert.encoder.layer.0.attention.self.value.bias',
 'bert.encoder.layer.0.attention.output.dense.weight',
 'bert.encoder.layer.0.attention.output.dense.bias',
 'bert.encoder.layer.0.attention.output.LayerNorm.weight',
 'bert.encoder.layer.0.attention.output.LayerNorm.bias',
 'bert.encoder.layer.0.intermediate.dense.weight',
 'bert.encoder.layer.0.intermediate.dense.bias',
 'bert.encoder.layer.0.output.dense.weight',
 'bert.encoder.layer.0.output.dense.bias',
 'bert.encoder.layer.0.output.LayerNorm.weight',
 'bert.encoder.layer

In [14]:
#将数据通过dataloader封装打包
train_dl = DataLoader(ds1['train'],shuffle=True,batch_size=16)
#创建模型，传入参数做分类记得要放id和label转换的接口
id2lbl = {i:k for i,k in enumerate(tags)}
lbl2id = {k:i for i,k in enumerate(tags)}
model2 = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                        num_labels=len(tags),
                                                        id2label = id2lbl,
                                                        label2id = lbl2id)
model2.to('cuda')
#模型参数分组
param_optimizer = list(model2.named_parameters())
bert_params,classifier_params = [],[]
#获取模型名称和参数，然后分别进行存储后指定学习率进行差分学习设置学习率
for name,params in param_optimizer:
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)
param_groups = [
    {'params':bert_params, 'lr':1e-5},
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3}
]

#AdamW有一股权重参数可以进行学习学习的参数以前都是直接用model.parameters这里可以用列表字典形式进行传入
optimizer = optim.AdamW(param_groups)
#学习率调度器(动态学习),transformers中提供的get_linear_schedule_with_warmup()
train_steps = len(train_dl)*5
scheduler = get_linear_schedule_with_warmup(optimizer,               #指定学习率的优化器
                                           num_warmup_steps=100,     #预热阶段的步数
                                           num_training_steps=train_steps)  #训练阶段的总步数

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print(torch.cuda.is_available())

True


In [18]:
for item in train_dl:
    print(item['input_ids'].shape, 
          item['token_type_ids'].shape, 
          item['attention_mask'].shape, 
          item['labels'].shape)
    break

torch.Size([16, 512]) torch.Size([16, 512]) torch.Size([16, 512]) torch.Size([16, 512])


### 差分学习 动态学习率 warmup训练

In [15]:
from tqdm import tqdm
DEVICE = 'cuda'

for epoch in range(5):
    model2.train()
    tpbar = tqdm(train_dl)   #一个batch一个batch的包拿过来循环对每个batch进行处理
    for item in tpbar:
        items = {k:v.to(DEVICE) for k,v in item.items()}  #这里就是tokenizer处理后的三个信息数值转成
        optimizer.zero_grad()
        logits = model2(**items)#**这里面会将item的几个key对应到模型中
        loss =logits.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        tpbar.set_description(f'Epoch:{epoch+1} ' + 
                  f'bert_lr:{scheduler.get_lr()[0]} ' + 
                  f'classifier_lr:{scheduler.get_lr()[1]} '+
                  f'Loss:{loss.item():.4f}')

Epoch:1 bert_lr:9e-06 classifier_lr:0.0009000000000000001 Loss:0.0668:   3%|▎         | 90/2813 [02:17<1:09:25,  1.53s/it]                  


RuntimeError: stack expects each tensor to be equal size, but got [512] at entry 0 and [710] at entry 13

In [17]:
print(len(train_dl))
for batch in train_dl:
    print("input_ids shape:", batch['input_ids'].shape)
    print("labels shape:", batch['labels'].shape)
    print("input_ids:", batch['input_ids'])
    print("labels:", batch['labels'])
    break  # 只查看第一个批次

2813
input_ids shape: torch.Size([16, 512])
labels shape: torch.Size([16, 512])
input_ids: tensor([[ 100, 1762,  686,  ...,    0,    0,    0],
        [2769,  812, 2792,  ...,    0,    0,    0],
        [ 686, 4518, 7942,  ...,    0,    0,    0],
        ...,
        [3315, 2237, 1300,  ...,    0,    0,    0],
        [1420, 1400, 8024,  ...,    0,    0,    0],
        [3219, 1921, 1762,  ...,    0,    0,    0]])
labels: tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [3, 4, 4,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


### 混合精度训练

In [25]:
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

train_dl = DataLoader(ds1['train'],shuffle=True,batch_size=16)
# 模型创建
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=7,
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model.to('cuda')

# 模型参数分组
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [],[]

for name,params in param_optimizer:
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {'params':bert_params, 'lr':1e-5},
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3}
]

# optimizer
optimizer = optim.AdamW(param_groups) # 优化器

# 学习率调度器
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=100, 
                                            num_training_steps=train_steps)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import torch
from tqdm import tqdm
DEVICE = 'cuda'

scaler =torch.GradScaler()

for epoch in range(5):
    pbar=tqdm(train_dl)
    for item in pbar:
        items = {k:v.to(DEVICE) for k,v in item.items()}
        optimizer.zero_grad()
        
        with torch.autocast(device_type='cuda'):
            logits = model(**items)
        loss = logits.loss
        # 缩放loss后，调用backward
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

    pbar.set_description(f'Epoch:{epoch+1} ' + 
          f'bert_lr:{scheduler.get_lr()[0]} ' + 
          f'classifier_lr:{scheduler.get_lr()[1]} '+
          f'Loss:{loss.item():.4f}')

  1%|          | 21/2813 [00:08<18:32,  2.51it/s]


RuntimeError: stack expects each tensor to be equal size, but got [512] at entry 0 and [699] at entry 6

### 分布式训练

In [3]:
%%writefile ddp_ner.py #jupyter notebook中的命令，作用是将 当前单元格中的代码写入指定的文件中

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from datasets import load_dataset

#1、设置分布式环境
def setup(rank,world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group('nccl',rank=rank,world_size=world_size)

#2、清理分布式环境
def cleanup():
    dist.destroy_process_group()

#3、定义循环训练
def train(rank,world_size):
    setup(rank,world_size)
    #加载数据
    ds = load_dataset('doushabao4766/msra_ner_k_V3')
    #entites index
    entites = ['O'] + list({'PER', 'LOC', 'ORG'})
    tags = ['O']
    for ent in entites[1:]:
        tags.append('B-' + ent)
        tags.append('I-' + ent)
    entite_index = [k:i for i,k in enumerate(entites)]

    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
    def data_input_proc(item):
        input_data = tokenizer(item['tokens'],
                              max_length=512,
                              truncation=True,
                              padding='max_length',
                              add_special_tokens=False,
                              is_split_into_words=True)
        input_data['labels'] = [tag + [0]*(512-len(tag)) for tag in item['ner_tags']]
        return input_data

    ds1 = ds.map(data_input_proc,batched=True) #batched == 1000
    
    local_rank = rank
            
    id2lbl = {i:tag for i, tag in enumerate(tags)}
    lbl2id = {tag:i for i, tag in enumerate(tags)}
    
    model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                            num_labels=21,
                                                            id2label=id2lbl,
                                                            label2id=lbl2id)
    model.to(local_rank)
    
    # 配置TrainingArguments 和 trainer
    args = TrainingArguments(
        output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
        num_train_epochs = 3,    # 训练 epoch
        save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
        per_device_train_batch_size=16,  # 训练批次设置
        per_device_eval_batch_size=16,   # 验证批次设置
        report_to='tensorboard',  # 训练输出记录
        eval_strategy="epoch",    #每个epoch结束后评估一次
        local_rank=local_rank,   # 当前进程 RANK
        fp16=True,               # 使用混合精度
        lr_scheduler_type='linear',  # 动态学习率, 线性平滑的学习率
        warmup_steps=100,        # 预热步数
        ddp_find_unused_parameters=False  # 优化DDP性能
    )

    # 目的是计算模型的性能指标，使用了 seqeval 库进行序列标注任务的评估
    def compute_metric(result):
        # result 是一个tuple (predicts, labels)
        
        # 评估对象 seqeval:是一个用于序列标注任务（命名实体），可以计算精确率（precision）、召回率recall和F1分值F1-score
        seqeval = evaluate.load('seqeval')
        predicts,labels = result
        predicts = np.argmax(prdicts, axis=2)
        
        # 准备评估数据
        predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        results = seqeval.compute(predictions=predicts, references=labels) # 获取评估结果：模型的性能指标
    
        return results

    # 用于处理序列标注任务的数据批处理 相当于dataloader
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)
    
    trainer = Trainer(
        model,
        args,   #训练参数，通常是一个 TrainingArguments 对象，包含学习率、批次大小、训练轮数等。。。
        train_dataset=ds2['train'],
        eval_dataset=ds2['validation'],
        data_collator=data_collator,     #数据批处理工具，这里使用 DataCollatorForTokenClassification
        compute_metrics=compute_metric   #评估函数，这里使用前面定义的 compute_metric 函数
    )
    
    trainer.train()

def main():
    world_size = torch.cuda.device_count() #计算GPU数量
    #mp.spawn(fn, args=(), nprocs=1, join=True, daemon=False)
    # fn：要启动的函数。这个函数会在每个进程中执行，args: fn 函数的额外参数,args=(world_size,) 表示将 world_size 作为参数传递给 train 函数,
    # 注意，参数必须是元组，所以即使只有一个参数，也需要在末尾加逗号，如 (world_size,)
    # nprocs:指定启动的进程总数，通常设置为可用GPU数量,join:主进程阻塞等待所有子进程结束,同步方式
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True) #mp包中用spawn创建线程，

if __name__ == "__main__":
    main()

UsageError: unrecognized arguments: #jupyter notebook中的命令，作用是将 当前单元格中的代码写入指定的文件中


In [2]:
!python ddp_ner.py

python3: can't open file '/kaggle/working/ddp_ner.py': [Errno 2] No such file or directory


### 利用训练后的模型推理

In [4]:
# 加载保存的模型和分词器
model = AutoModelForTokenClassification.from_pretrained("ner_model")
tokenizer = AutoTokenizer.from_pretrained("ner_model")

# 假设有一个测试样本
test_sample = {
    "tokens": ["北", "京", "市", "好"]
}

# 对测试样本进行分词
inputs = tokenizer(
    test_sample["tokens"],
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding="max_length",
    add_special_tokens=False,
    is_split_into_words=True
)

# 模型推理
with torch.no_grad():
    outputs = model(**inputs)

# 获取预测标签
predictions = torch.argmax(outputs.logits, dim=2).numpy()

# 转换预测结果为标签
predicted_labels = [id2lbl[p] for p in predictions[0]]

print("Predicted labels:", predicted_labels)

OSError: ner_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [41]:
for sample in ds1['train']:
    assert len(sample['input_ids']) == len(sample['labels']), "输入和标签长度不一致!"
# 检查数据集中每个样本的长度
# for sample in ds1['train']:
#     if assert len(sample['input_ids']) != len(sample['labels']):
#         print(len(sample['input_ids']))
#         print( len(sample['labels']))
        

    # print("input_ids length:", len(sample['input_ids']))
    # print("labels length:", len(sample['labels']))
    # # 如果发现长度不一致，打印出该样本
    # if len(sample['input_ids']) != len(sample['labels']):
    #     print("Mismatched lengths found!")
    #     print("input_ids:", sample['input_ids'])
    #     print("labels:", sample['labels'])
    #     break

AssertionError: 输入和标签长度不一致!

In [22]:
token_index = tokenizer.encode('2000年2月add', add_special_tokens=False)
print(token_index)
tokens = tokenizer.decode(token_index)
print(tokens)

[8202, 2399, 123, 3299, 10253]
2000 年 2 月 add


In [26]:
input_data = tokenizer([list('2000年2月add')], add_special_tokens=False, truncation=True, 
                       is_split_into_words=True)
print(input_data)
#中文Bert分词在日期时间和英文转换上会合并如上，影响ner的准确性，
#所以用将文本拆分为list就变成单个词了需要在tokenizer中属性: is_split_into_words=True，这样可以一一对应如下。

tokens = tokenizer.decode(token_index)
print(tokens) # 返回token对应逐个字符

{'input_ids': [[123, 121, 121, 121, 2399, 123, 3299, 143, 146, 146]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
2000 年 2 月 add


In [29]:
for item in ds1['train']:
    print(len(item['input_ids']), len(item['token_type_ids']), len(item['attention_mask']), len(item['labels']))
    break

50 50 50 50
